In [27]:
import os
import requests
import re
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from io import StringIO
from WebScrapingProject.Schools.ai_utils import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
TABLE = 'Tuition'

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
url = 'https://www.ciam.edu/cost-aid#TuitionandFees'
page = requests.get(url, headers=headers)

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
content = ''

start = soup.find(string='Tuition & Fees')
end = soup.find(string='Application Deadlines')

if start and end:  # ensuring both start and end points are found
    for element in start.parent.find_all_next(string=True):  # iterate over next elements in tree
        if element == end:
            break  # stop iteration when reaching 'end' string
        if isinstance(element, NavigableString):
            content += element.strip() + '\n'  # add a newline character after each element


content = re.sub('\n+', '\n', content)
content = content.replace(",", "")

In [18]:
print(content)

Tuition & Fees
Estimated Tuition Rates 2022-2023
This chart estimates the cost of attending CIAM for two years as a graduate student. For a significant proportion of our students these expenses are offset by scholarships and aid. Keep in mind that your total cost may vary depending on your personal expenses and the degree or concentration you choose.
Master's of Business Administration in Executive Management
Tuition
Full-Time Status (2 courses per term)
$4000 per term
Part-Time Status (1 course per term)
$2000 per term
Tution Costs Per Credit
$667
Total Tuition:
$24000 for entire program
Student Fees 2022-2023
Category:
Fees:
Application Fee (Non-Refundable)
$75
STRF Fee (Non-Refundable)
$60
Estimated Total Textbook Cost
No Additional Cost
Estimated Parking Fee
No Additional Cost
INT/CPT Registration Fee
$100*
Total Estimated Program Associated Fees:
$235
*Students are required to successfully complete INT501 & INT599 to graduate with a $50 registration fee for each course; internatio

In [21]:
output_text = get_confident_response(
        extract_tuition_to_csv(content),
        model='gpt-4-0125-preview',
        temperature=0.4,
        identical_responses_needed=2,
        max_attempts=10
    )

output_text = output_text.lstrip("```").rstrip("```")  # This removes backticks from start and end
output_text = output_text.strip()  # This removes leading and trailing newlines

[2024-04-16 16:19:14,272] 12 WebScrapingProject.Schools.ai_utils - DEBUG - Returning prompt: I have extracted tuition fee details from a webpage. The information can include program name, tuition per credit or per term, additional applicable fees, and the estimated overall cost of attendance. Prepare a CSV-like output that represents these categories of information, using an appropriate format based on the structure of the provided data. Each entry should be listed on a new line with elements separated by commas. All currency amounts should be represented in 'USD' format. Avoid any additional bibliographical conversation or content in the response. Here is the obtained text: 
'Tuition & Fees
Estimated Tuition Rates 2022-2023
This chart estimates the cost of attending CIAM for two years as a graduate student. For a significant proportion of our students these expenses are offset by scholarships and aid. Keep in mind that your total cost may vary depending on your personal expenses and t

In [22]:
data = StringIO(output_text)
df = pd.read_csv(data, sep=',')

In [23]:
df

,Program Name,Tuition Status,Tuition Per Term,Tuition Per Credit,Total Tuition,Application Fee,STRF Fee,Textbook Cost,Parking Fee,INT/CPT Registration Fee,Total Estimated Program Associated Fees,Total Estimated Program Costs
0,Master's of Business Administration in Executive Management,Full-Time,$4000,USD 667,$24000,USD 75,USD 60,No Additional Cost,No Additional Cost,USD 100,USD 235,USD 24235
1,Master's of Business Administration in Executive Management,Part-Time,$2000,USD 667,$24000,USD 75,USD 60,No Additional Cost,No Additional Cost,USD 100,USD 235,USD 24235


In [24]:
df.to_csv('test.csv')

In [29]:
command = f"jupyter nbconvert --to script {TABLE}"

os.system(command)

[NbConvertApp] Converting notebook Tuition.ipynb to script
[NbConvertApp] Writing 2791 bytes to Tuition.py


0